In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.express as px

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Importation du jeu de données

In [ ]:
df_2018 = pd.read_csv('valeursfoncieres-2018.txt', sep="|", decimal=",")
df_2019 = pd.read_csv('valeursfoncieres-2019.txt', sep="|", decimal=",")
df_2020 = pd.read_csv('valeursfoncieres-2020.txt', sep="|", decimal=",")
df_2021 = pd.read_csv('valeursfoncieres-2021.txt', sep="|", decimal=",")

In [ ]:
df_2018.shape[0] + df_2019.shape[0] + df_2020.shape[0] + df_2021.shape[0]

In [ ]:
df = pd.concat([df_2018,df_2019, df_2020, df_2021], axis=0)

In [ ]:
df.shape

In [ ]:
# On paramétrise notre display function pour avoir toutes les variables
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.loc[df["Code departement"] == 67]

In [ ]:
df.loc[df["Code departement"] == 68]

In [ ]:
df.loc[df["Code departement"] == 57]

In [ ]:
# On extrait les noms des variables
df.columns

In [ ]:
# On regarde le nombre de valeurs unique dans chaque colonne pour enlever les variables qui n'apporte pas d'information
for col in df.columns:
    print(col)
    print(len(df[col].unique()))

# Nettoyage du jeu de données

In [ ]:
# On enlève donc les variables avec une valeur (souvent NaN) ou d'identifiant (qui n'auront donc pas d'intérêt dans la prédiction ou l'exploration)
df.drop(columns=['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI','No disposition', 'No plan', 'Identifiant local', 'No Volume'], inplace=True)

In [ ]:
df.head()

B/T/Q représente l'indice de répétition qui est est une mention qui complète une numérotation de voirie. L’indice de répétition permet de
différencier plusieurs adresses portant le même numéro dans la même rue. Elle ne nous servira pas en soit car donc pas car 

In [ ]:
df.drop(columns=["B/T/Q"], inplace=True)

In [ ]:
df.head(n= 10)

In [ ]:
# On regarde le nombre de valeurs manquantes dans chaque colonne pour enlever les variables qui n'apporte pas d'information
for col in df.columns:
    print(col)
    print(df[col].isnull().sum()/df.shape[0])

Le nombre de lot est suffisant pour nous indiquer la présence ou non de lots. Le reste est redondant.

In [ ]:
df.drop(columns=['1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot'], inplace=True)

In [ ]:
df.head(n=10)

Nous nous occupons désormais de la variable nature culture spéciale qui nous indique la présence ou non d'un extérieur

In [ ]:
df.head()

In [ ]:
df.drop(columns=[ "Nature culture speciale"], inplace=True)

In [ ]:
df.head(n=10)

In [ ]:
df.loc[df["Nature culture"].isna(), "exterieur"] = 0
df.loc[df["exterieur"].isna(), "exterieur"] = 1

df.head()

In [ ]:
# La présence d'un terrain est clairement indiquer par la surface présente, donc si elle est de 0: il n'y a pas de terrain
df.loc[df["Surface terrain"].isnull(), "Surface terrain"] = 0

In [ ]:
df.head(n=10)

In [ ]:
# En s'appuyant sur la documentation, on se rend compte que le relatif aux sections n'est pas pertinent
df.drop(columns=["Prefixe de section", "Section"], inplace=True)

In [ ]:
df.head()

In [ ]:
# On extrait les différentes modalités prises par Nature mutation
df["Nature mutation"].unique()

In [ ]:
# On observe que ce n'est pas pertinent pour notre problématique.
df.drop(columns=["Nature mutation"], inplace=True)

In [ ]:
df.head()

La variable en relation avec l'adresse se sépare en plusieurs variables, nous allons donc la concaténer. Si l'adresse est nulle, c'est parce qu'il n'y a pas de n° de voie (lieux dits?), donc on remplie adresse avec "Voie".

In [ ]:
df["key"]=  df["No voie"].astype(str)+df["Voie"]+df["Commune"]+df["Date mutation"]
df.head(n=10)

In [ ]:
df.head()

In [ ]:
# On enlève les références à l'adresse qui ne nous seront d'aucunes utilité
df.drop(columns=["Type de voie", "Voie", "No voie", "Code voie"], inplace=True)

In [ ]:
df.head(n=10)

In [ ]:
# On enlève également le code département qui ne nous servira pas pour récupérer les départements (nous utiliserons le code postal et le nom de la commune)
df.drop(columns=["Code departement"], inplace=True)

In [ ]:
df.head()

In [ ]:
# Pour ne garder les ventes que de lots uniques, on enlève toutes les ventes qui se répètent plus d'une fois
df2 = df.drop_duplicates(subset='key', keep=False)

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
# On change le format de la date de vente en format datetime.
df2.loc[:, 'Date mutation'] = pd.to_datetime(df2['Date mutation'] , format='%d/%m/%Y')

In [ ]:
# On récupère le mois et l'année de la vente.
df2.loc[:, 'year'] = pd.DatetimeIndex(df2['Date mutation']).year
df2.loc[:, 'month'] = pd.DatetimeIndex(df2['Date mutation']).month

In [ ]:
df2.head()

In [ ]:
# On enlève la clé qui nous a servi à enlever les duplicatas et la date entière qui ne nous sert plus
df2.drop(columns=["Date mutation", "key"], inplace=True)

In [ ]:
df2.head()

In [ ]:
# Si NaN values dans ces variables de surfaces ou nombre de pièces principales, 0 assigné
df2.loc[df2["Surface reelle bati"].isnull(), "Surface reelle bati"] = 0
df2.loc[df2["Surface terrain"].isnull(), "Surface terrain"] = 0
df2.loc[df2["Nombre pieces principales"].isnull(), "Nombre pieces principales"] = 0

In [ ]:
# On enlève la clé qui nous a servi à enlever les duplicatas et la date entière qui ne nous sert plus
df2.drop(columns=["Nature culture"], inplace=True)

In [ ]:
# On enlève la clé qui nous a servi à enlever les duplicatas et la date entière qui ne nous sert plus
df2.drop(columns=["Code type local"], inplace=True)

In [ ]:
df2.head()

In [ ]:
df2.loc[df2["Type local"].isna(), "Type local"] = "Autres"

In [ ]:
df2.head()

In [ ]:
for col in df2.columns:
    print(col)
    print(df2[col].isnull().sum())

In [ ]:
# On enlève les lignes avec des valeurs nulles
df2 = df2.dropna()

In [ ]:
for col in df2.columns:
    print(col)
    print(df2[col].isnull().sum())

Certains biens sont à l'euro symbolique ou ont un prix inférieur à 100€, nous enlevons ces biens

In [ ]:
# On visualise le pourcentage de biens avec un prix de vente inférieur à 100€.
euro_symbol = (df2["Valeur fonciere"] < 100).value_counts()

plt.pie(euro_symbol, autopct='%.0f%%')

In [ ]:
# On ne garde que les biens ayant eu un prix de vente supérieur à 100€.
df3 = df2[df2["Valeur fonciere"] > 100]
df3.head()

In [ ]:
# On récupère les différents quartiles pour pouvoir enlever les biens avec un prix anormalement hauts comparés aux autres biens.
Q1, Q2, Q3 = df3['Valeur fonciere'].quantile([0.25, 0.5, 0.75])

# On calcule le rang interquartile
IQR = Q3-Q1

In [ ]:
# On détexte les biens ayant un prix supérieur au troisième quartile multiplié par 1.5 fois le rang interquartile
outliers_max = df3["Valeur fonciere"] > (Q3 + 1.5*(IQR))
plt.pie(outliers_max.value_counts(), autopct='%.0f%%')

In [ ]:
df3.shape

In [ ]:
# On ne garde que les biens n'ayant pas un prix de vente anormalement haut.
df3 = df3[df3["Valeur fonciere"] < (Q3 + 1.5*(IQR))]

# Liaison avec données des départements et communes

In [ ]:
# On récupère la base de données des communes/département
departement = pd.read_csv("communes-departement-region.csv")
departement.head()

In [ ]:
# On ne garde que les variables qui nous serviront à cartographier nos données
departement  = departement.loc[:,["code_postal", "nom_commune", "nom_departement","nom_region", "latitude", "longitude"]]
# On met en majuscule le nom des communes pour joindre les bases de données
departement["nom_commune"] = departement["nom_commune"].str.upper()
departement.head()

In [ ]:
df3.shape

In [ ]:
# On fait une jointure interne entre les deux bases de données
df4  = pd.merge(df3, departement, left_on=['Commune', 'Code postal'], right_on=['nom_commune', 'code_postal'], how='inner')
df4.head()

In [ ]:
df4.shape

In [ ]:
# On change le type de pièces principales pour la visualisation
df4["Nombre pieces principales"] = df4["Nombre pieces principales"].astype(int)

In [ ]:
# On raccourcit le nom de ce type de local pour la visualisation
df4.loc[df4["Type local"] == 'Local industriel. commercial ou assimilé', "Type local"] = "Local"

In [ ]:
fig, axes = plt.subplots(2, 3,figsize=(30,12))

#create boxplot in each subplot
sns.boxplot(data=df4, x="month", y="Valeur fonciere", ax=axes[0,0])
sns.boxplot(data=df4, x="Type local", y="Valeur fonciere", ax=axes[0,1])
sns.boxplot(data=df4, x="Nombre de lots", y="Valeur fonciere", ax=axes[0,2])
sns.boxplot(data=df4, x="Nombre pieces principales", y="Valeur fonciere", ax=axes[1,0])
sns.scatterplot(data=df4, x="Surface reelle bati", y="Valeur fonciere", ax=axes[1,1])
sns.scatterplot(data=df4, x="Surface terrain", y="Valeur fonciere", ax=axes[1,2])

In [ ]:
month_dict_fr = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai',
    6: 'Juin',
    7: 'Juillet',
    8: 'Août',
    9: 'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}


In [ ]:
df4["month"] = [month_dict_fr[month] for month in df4["month"]]

In [ ]:
order_month = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']


In [ ]:
fig = px.box(df4, x="month", y="Valeur fonciere", category_orders={'month': order_month}, color="month")

fig.show()

In [ ]:
pivot_departement = df4.pivot_table(index=["nom_departement","month"], values="Valeur fonciere", aggfunc="mean").reset_index()
pivot_departement

In [ ]:
months_order = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']

# Create a categorical data type with the order of months
month_cat = pd.CategoricalDtype(categories=months_order, ordered=True)

# Convert the 'month' column to the categorical data type
pivot_departement['month'] = pivot_departement['month'].astype(month_cat)

# Now sort by the 'month' column
pivot_departement = pivot_departement.sort_values(by='month')

In [ ]:

fig = px.line(pivot_departement, x="month", y="Valeur fonciere", color='nom_departement')
fig.show()

In [ ]:
pivot_region = df4.pivot_table(index=["nom_region","month"], values="Valeur fonciere", aggfunc="mean").reset_index()

# Convert the 'month' column to the categorical data type
pivot_region['month'] = pivot_region['month'].astype(month_cat)

# Now sort by the 'month' column
pivot_region = pivot_region.sort_values(by='month')

In [ ]:

fig = px.line(pivot_region, x="month", y="Valeur fonciere", color='nom_region')
fig.show()

In [ ]:
fig = plt.subplots(1, 1,figsize=(30,12))
sns.boxplot(data=df4, x="nom_departement", y="Valeur fonciere")
plt.xticks(rotation=45, ha='right')


In [ ]:
for region in df4["nom_region"].unique():
    print(region)

In [ ]:
# Pour chaque region, on construit le boxplot des départements correspondant: 
for region in df4["nom_region"].unique():
    df_graph = df4.loc[df4["nom_region"] == region]
    fig = px.box(df_graph, x="nom_departement", y="Valeur fonciere",  color="nom_departement",  title=region)
    fig.update_layout(showlegend=False)
    fig.show()

In [ ]:
fig = plt.subplots(1, 1,figsize=(20,12))
sns.boxplot(data=df4, x="nom_region", y="Valeur fonciere")
plt.xticks(rotation=45, ha='right')

In [ ]:
fig = px.box(df4, x="nom_region", y="Valeur fonciere",  color="nom_region")

fig.show()

# Map

In [ ]:
df4.head()

In [ ]:
mean_values = pd.pivot_table(data=df4, index=["longitude", "latitude"], values="Valeur fonciere", aggfunc="mean")
mean_values = mean_values.reset_index()
mean_values

In [ ]:
mean_values.head()

In [ ]:
mean_values['normalized_value'] = (mean_values['Valeur fonciere'] -mean_values['Valeur fonciere'].min()) / (mean_values['Valeur fonciere'].max() - mean_values['Valeur fonciere'].min())
mean_values['normalized_value'].plot.hist(bins=30, alpha=0.5)


In [ ]:
fig = px.scatter_geo(mean_values,
                     lat='latitude',
                     lon='longitude',
                     scope='europe', 
                    color='Valeur fonciere',
                     color_continuous_scale='Inferno',
                     # Centré sur la france
                     center=dict(lat=46.26, lon=2.52),
                     # Normalisé car doit être en 0 et 1
                     opacity=mean_values["normalized_value"])

#On update la map pour la centrer sur la france
fig.update_layout(
    autosize=True,
    height=600,
    geo=dict(
        center=dict(
            lat=46.26,
            lon=2.52
        ),
        scope='europe',
        projection_scale=6
    )
)
fig.show()

### Polygons de DataGouv: https://www.data.gouv.fr/fr/datasets/contours-des-communes-de-france-simplifie-avec-regions-et-departement-doutre-mer-rapproches/

In [ ]:
import json

In [ ]:
mean_values_json = pd.pivot_table(data=df4, index=["nom_region"], values="Valeur fonciere", aggfunc="mean")
mean_values_json = mean_values_json.reset_index()
mean_values_json

In [ ]:
mean_values_json.loc[mean_values_json["nom_region"] == "Nouvelle-Aquitaine", "nom_region"] = "Nouvelle Aquitaine"
mean_values_json.loc[mean_values_json["nom_region"] == "Grand Est", "nom_region"] = "Grand-Est"

In [ ]:
with open("regions.json", "r") as file:
    regions_data = json.load(file)

In [ ]:
# Create the choropleth map
fig = px.choropleth(
    mean_values_json,  # replace df with your DataFrame
    geojson=regions_data,
    locations='nom_region',  # replace 'id' with the column name containing the regions' ids
    color='Valeur fonciere',  # replace 'value' with the column name containing the values you want to plot
    color_continuous_scale='Inferno',
    featureidkey="properties.libgeo",  # replace 'properties.id' with the path to the ids in the geojson
    range_color=[100000, 250000]
)

# Update the geos layout to focus on France
fig.update_geos(
    center={"lat": 46.603354, "lon": 1.888334},  # Coordinates of France's centroid
    projection_scale=15,  # Adjust the scale to fit France
    visible=False  # Hide the base map
)

# Update the layout
fig.update_layout(
    title="Choropleth Map of France",
    margin={"r":0,"t":40,"l":0,"b":0}
)

# Show the map
fig.show()

In [ ]:
with open("departement.json", "r") as file:
    dep_data = json.load(file)

In [ ]:
mean_values_json_departement = pd.pivot_table(data=df4, index=["nom_departement"], values="Valeur fonciere", aggfunc="mean")
mean_values_json_departement = mean_values_json_departement.reset_index()
mean_values_json_departement

In [ ]:
# Create the choropleth map
fig = px.choropleth(
    mean_values_json_departement,  # replace df with your DataFrame
    geojson=dep_data,
    locations='nom_departement',  # replace 'id' with the column name containing the regions' ids
    color='Valeur fonciere',  # replace 'value' with the column name containing the values you want to plot
    color_continuous_scale='Inferno',
    featureidkey="properties.libgeo"  # replace 'properties.id' with the path to the ids in the geojson
)

# Update the geos layout to focus on France
fig.update_geos(
    center={"lat": 46.603354, "lon": 1.888334},  # Coordinates of France's centroid
    projection_scale=15,  # Adjust the scale to fit France
    visible=False  # Hide the base map
)

# Update the layout
fig.update_layout(
    title="Choropleth Map of France",
    margin={"r":0,"t":40,"l":0,"b":0}
)

# Show the map
fig.show()

# Variables supplémentaires

### Population active par département: https://www.insee.fr/fr/statistiques/2012710#titre-bloc-1

In [ ]:
df4.head()

In [ ]:
pop = pd.read_excel("pop_active.xlsx")
pop.head()

In [ ]:
df4.shape

In [ ]:
# On fait une jointure interne entre les deux bases de données
df5  = pd.merge(df4, pop, on="nom_departement", how='inner')
df5.head()

### Salaire Net Horaire Moyen par département (2021)- https://www.insee.fr/fr/statistiques/2021266

In [ ]:
# On importe le salaire horaire moyen
salaire = pd.read_excel("base-cc-bases-tous-salaries-2021.xlsx")
salaire.head()

In [ ]:
# On fait une jointure interne entre les deux bases de données
df5  = pd.merge(df5, salaire, on="nom_departement", how='inner')
df5.head()

In [ ]:
# On crée une matrice de corrélation
matrix = df5.loc[:,['Valeur fonciere', 'Nombre de lots', 'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain', "pop_active", "salaire_moyen"]].corr()
matrix

# Nombre d'écoles par département
https://www.observatoire-des-territoires.gouv.fr/nombre-decoles-elementaires

In [ ]:
# On importe le dataset avec le nombre d'écoles élémentaires par commune
ecole = pd.read_excel("ecoles2.xlsx")
ecole.head()

In [ ]:
# On fait une jointure interne entre les deux bases de données
df6  = pd.merge(df5, ecole, on="nom_departement", how='inner')
df6.head()

# Modélisation